# **Diabetic Patient Readmission -- Data Wrangling**

This dataset was analyzed by numerous Virginia Commonwealth University faculty in a recent research article which is accompanied by feature descriptions. These can be found at https://www.hindawi.com/journals/bmri/2014/781670/tab1/.

In [1]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
df1 = pd.read_csv('diabetic_data.csv')
df2 = pd.read_csv('IDs_mapping.csv')

**Data Collection:**

In [3]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [4]:
print(df2.head(10))
print(df2.info())

          admission_type_id    description
0                         1      Emergency
1                         2         Urgent
2                         3       Elective
3                         4        Newborn
4                         5  Not Available
5                         6            NaN
6                         7  Trauma Center
7                         8     Not Mapped
8                       NaN            NaN
9  discharge_disposition_id    description
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   admission_type_id  65 non-null     object
 1   description        62 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None


**Data Organization:**

In [5]:
print(df2[df2.description == 'description'])

           admission_type_id  description
9   discharge_disposition_id  description
41       admission_source_id  description


The 'IDs_mapping.csv' seems to contain *multiple* tables containing key-value descriptions for designated IDs in the primary dataset.

In [6]:
admission_type_ids = df2[:8] # Adjusted from [:9] to remove NaN ID
discharge_disposition_ids = df2[10:40] # Adjusted from [10:41] to remove NaN ID
discharge_disposition_ids = discharge_disposition_ids.reset_index(drop=True).rename(columns={'admission_type_id':'discharge_disposition_id'})
admission_source_ids = df2[42:]
admission_source_ids = admission_source_ids.reset_index(drop=True).rename(columns={'admission_type_id':'admission_source_id'})
print(admission_type_ids)
print(discharge_disposition_ids)
print(admission_source_ids)

  admission_type_id    description
0                 1      Emergency
1                 2         Urgent
2                 3       Elective
3                 4        Newborn
4                 5  Not Available
5                 6            NaN
6                 7  Trauma Center
7                 8     Not Mapped
   discharge_disposition_id                                        description
0                         1                                 Discharged to home
1                         2  Discharged/transferred to another short term h...
2                         3                      Discharged/transferred to SNF
3                         4                      Discharged/transferred to ICF
4                         5  Discharged/transferred to another type of inpa...
5                         6  Discharged/transferred to home with home healt...
6                         7                                           Left AMA
7                         8  Discharged/transferred t

As we have a singular, primary dataset that we will work with, there is no need for merging/joining at this stage. Introducing these descriptions into the dataset would likely complicate future analysis and modeling.

In [7]:
# Saving individual id libraries for future reference
admission_type_ids.to_csv('admission_type_ids.csv', index=False)
discharge_disposition_ids.to_csv('discharge_disposistion_ids.csv', index=False)
admission_source_ids.to_csv('admission_source_ids.csv', index=False)

**Data Definition:**

For feature/column definitions/descriptions, see https://www.hindawi.com/journals/bmri/2014/781670/tab1/.

In [8]:
df1.keys()

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [9]:
df1[['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'A1Cresult', 'insulin', 'change', 'readmitted']]

,encounter_id,patient_nbr,weight,payer_code,A1Cresult,insulin,change,readmitted
0,2278392,8222157,?,?,None,No,No,NO
1,149190,55629189,?,?,None,Up,Ch,>30
2,64410,86047875,?,?,None,No,No,NO
3,500364,82442376,?,?,None,Up,Ch,NO
4,16680,42519267,?,?,None,Steady,Ch,NO
...,...,...,...,...,...,...,...,...
101761,443847548,100162476,?,MC,>8,Down,Ch,>30
101762,443847782,74694222,?,MC,None,Steady,No,NO
101763,443854148,41088789,?,MC,None,Down,Ch,NO
101764,443857166,31693671,?,MC,None,Up,Ch,NO


In [10]:
df1[['encounter_id','A1Cresult', 'diabetesMed', 'readmitted', 'diag_1', 'diag_2', 'diag_3']]

,encounter_id,A1Cresult,diabetesMed,readmitted,diag_1,diag_2,diag_3
0,2278392,None,No,NO,250.83,?,?
1,149190,None,Yes,>30,276,250.01,255
2,64410,None,Yes,NO,648,250,V27
3,500364,None,Yes,NO,8,250.43,403
4,16680,None,Yes,NO,197,157,250
...,...,...,...,...,...,...,...
101761,443847548,>8,Yes,>30,250.13,291,458
101762,443847782,None,Yes,NO,560,276,787
101763,443854148,None,Yes,NO,38,590,296
101764,443857166,None,Yes,NO,996,285,998


'diag_1', 'diag_2', and 'diag_3' are correspond to ICD-9-CM (Version 32) codes for patients' primary, secondary, and additional secondary diagnoses, respectively. This reference can be found here: https://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/codes<br><br>
The initial dataset was stipulated to include only patients with a previous diabetes mellitus diagnosis (ICD9: 250.xx).<br><br>
**Generated a .html report which can be used to explore df1 in greater detail using pandas-profiler and the following code:**<br>
import pandas_profiling<br>
from pandas_profiling.utils.cache import cache_file<br>
profile_report = df1.profile_report(explorative=True, html={'style': {'full_width': True}})<br>
profile_report.to_file('df1_profile.html')<br>

In [12]:
df1.weight.describe()

count     101766
unique        10
top            ?
freq       98569
Name: weight, dtype: object

In [13]:
print("We only have", 101766-98569,'out of 101766 weights which equates to',100*98569/101766,'% missingness.')

We only have 3197 out of 101766 weights which equates to 96.85847925633315 % missingness.


In [14]:
df1.diabetesMed.describe()

count     101766
unique         2
top          Yes
freq       78363
Name: diabetesMed, dtype: object

We may filter in later stages for patient encounters that involve a change in diabetes medication.

In [15]:
df1.readmitted.describe()

count     101766
unique         3
top           NO
freq       54864
Name: readmitted, dtype: object

Observations made in the df1.html profile:<br>
- 'time_in_hospital' distribution is right-tailed, with a range of 1-14 (days) and a mean of 4.40.
- 'payer_code' contains 39.6% '?'s. This column is removed to avoid errors.
- 'medical_specialty' contains 49.1% '?'s. This column is kept, but imputed 'missing' for '?'.
- 'num_lab_procedures' has a fairly normal distribution, though 1 procedure is the most frequent at 3.2% frequency with 43 procedures at 2.8% as the second most-frequent. The distribution has a mean of 43.10 and a range of 1-132.
- 'num_procedures' distribution is right-tailed, with a 'bump' at the max value of 6.
- 'num_medications' distribution is right-tailed, with a range of 1-81 and mean of 16.02.
- 'diag_' columns contain a high number of ICD-9 codes starting with 428, 276, 427, 250, and 401.
- 'number_diagnoses' is particularly interesting, with 48.6% of values at 9, a range of 1-16, and a mean of 7.42.
- 'A1Cresult' is 83.3% 'None', meaning the vast majority of these encounters do not include the taking of an HbA1c measurement.
- 'examide' and 'citoglipton' medications were unchanged across the dataset and will therefore be dropped in our 'clean' dataset.
- 'insulin' contains 46.6% 'No', indicating ~47% of the encounters do not involve patients who are actively taking insulin.
- 'change' column contains 53.8% 'No', so a slight majority of these encounters involve patients whose diabetes medication is unchanged (dosage or generic name) afterwards.
- 'diabetesMed' column shows 77.0% of encounters involve a diabetes medication prescription.

**Data Cleaning:**<br>

**From the article:** "The preliminary dataset contained multiple inpatient visits for some patients and the observations could not be considered as statistically independent, an assumption of the logistic regression model. We thus used only one encounter per patient; in particular, we considered only the first encounter for each patient as the primary admission and determined whether or not they were readmitted within 30 days." <br><br>
Executed below:

In [16]:
df1 = df1[df1['number_inpatient'] == 0]
df1 = df1.drop(columns='number_inpatient')
print(df1.shape)
df1.head()

(67630, 49)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30


In [17]:
no_death_filter = ['13','14','18','19','20','21']
drops = discharge_disposition_ids[discharge_disposition_ids.discharge_disposition_id.isin(no_death_filter)]
drops

,discharge_disposition_id,description
12,13,Hospice / home
13,14,Hospice / medical facility
17,18,NaN
18,19,"Expired at home. Medicaid only, hospice."
19,20,"Expired in a medical facility. Medicaid only, ..."
20,21,"Expired, place unknown. Medicaid only, hospice."


In [18]:
no_death_filter = [13,14,18,19,20,21]
df1 = df1[~df1.discharge_disposition_id.isin(no_death_filter)]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64781 entries, 0 to 101765
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id              64781 non-null  int64 
 1   patient_nbr               64781 non-null  int64 
 2   race                      64781 non-null  object
 3   gender                    64781 non-null  object
 4   age                       64781 non-null  object
 5   weight                    64781 non-null  object
 6   admission_type_id         64781 non-null  int64 
 7   discharge_disposition_id  64781 non-null  int64 
 8   admission_source_id       64781 non-null  int64 
 9   time_in_hospital          64781 non-null  int64 
 10  payer_code                64781 non-null  object
 11  medical_specialty         64781 non-null  object
 12  num_lab_procedures        64781 non-null  int64 
 13  num_procedures            64781 non-null  int64 
 14  num_medications      

In [19]:
df1['encounter_id'].nunique()

64781

In [20]:
df1.patient_nbr.nunique()

61056

Dropping constants 'examide' and 'citoglipton', as well as the sparsely-recorded 'weight' and 'payer_code' columns:

In [21]:
df1 = df1.drop(columns=['examide','citoglipton','weight','payer_code'])
df1.shape

(64781, 45)

The 'encounter_id' column seems to be an ideal unique identifier for encounters.

The 'patient_nbr' column also appears to be a unique identifier for individual patients, with which multiple encounters are sometimes recorded in this dataset.

The data is in a good enough state to move forward and handle further data cleaning as deemed necessary by the requirements of the modeling stage.<br>
<br>
The provided link to feature descriptions also shows calculated missingness percentages.

In [22]:
df1['medical_specialty'].describe()

count     64781
unique       71
top           ?
freq      30841
Name: medical_specialty, dtype: object

In [23]:
df1.medical_specialty = df1.medical_specialty.replace('?','missing')

In [24]:
df1.readmitted = df1.readmitted.replace(['NO', '>30'], 'Other')
df1.readmitted.value_counts()

Other    59326
<30       5455
Name: readmitted, dtype: int64

In [25]:
df1.loc[:,'diag_1':'diag_3'] = df1.loc[:,'diag_1':'diag_3'].replace('?','missing')

In [26]:
df1.head(10)

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,Other
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,missing,...,No,Up,No,No,No,No,No,Ch,Yes,Other
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,missing,...,No,Up,No,No,No,No,No,Ch,Yes,Other
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,missing,...,No,Steady,No,No,No,No,No,Ch,Yes,Other
5,35754,82637451,Caucasian,Male,[50-60),2,1,2,3,missing,...,No,Steady,No,No,No,No,No,No,Yes,Other
6,55842,84259809,Caucasian,Male,[60-70),3,1,2,4,missing,...,No,Steady,No,No,No,No,No,Ch,Yes,Other
7,63768,114882984,Caucasian,Male,[70-80),1,1,7,5,missing,...,No,No,No,No,No,No,No,No,Yes,Other
8,12522,48330783,Caucasian,Female,[80-90),2,1,4,13,missing,...,No,Steady,No,No,No,No,No,Ch,Yes,Other
9,15738,63555939,Caucasian,Female,[90-100),3,3,4,12,InternalMedicine,...,No,Steady,No,No,No,No,No,Ch,Yes,Other
10,28236,89869032,AfricanAmerican,Female,[40-50),1,1,7,9,missing,...,No,Steady,No,No,No,No,No,No,Yes,Other


In [27]:
df1.to_csv('clean_data.csv', index=False)